In [3]:
import h5py
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random

In [4]:
fname_long_term = 'E:\Dima\PhD\Papers\data\states\states_model_80_1_80_500_long_term.h5py'
fname_original = 'E:\Dima\PhD\Papers\data\states\states_model_80_1_80_500.h5py'
tokens_file = 'E:/Dima/PhD/Papers/data/states/tokens_index_model_80_1_80_500_long_term.csv'

In [5]:
m = h5py.File(fname_long_term, 'r')
f = h5py.File(fname_original, 'r')
df_tokens = pd.read_csv(tokens_file)
df_tokens.head()

,id,token,long_term_token
0,0,preface supposing that truth is a womanwhat ...,preface supposing that truth is a womanwhat ...
1,1,reface supposing that truth is a womanwhat t...,reface supposing that truth is a womanwhat t...
2,2,eface supposing that truth is a womanwhat th...,eface supposing that truth is a womanwhat th...
3,3,face supposing that truth is a womanwhat the...,face supposing that truth is a womanwhat the...
4,4,ace supposing that truth is a womanwhat then...,ace supposing that truth is a womanwhat then...


In [6]:
p_long_term = max(m['y_' + str(20)][()])

In [7]:
p_original = max(f['y_' + str(20)][()])

In [8]:
k = 0
prob_original_list = []
prob_long_term_list = []
p5_satisfies = []

for elem_id, row in df_tokens.iterrows():
    elem = row[1]  # long term string
    p_long_term = max(m['y_' + str(elem_id)][()])
    p_original = max(f['y_' + str(elem_id)][()])
    
    if (p_original > 0.7) & (p_long_term > 0.65):
        p5_satisfies.append(1)
    else:
        p5_satisfies.append(0)
    
    prob_long_term_list.append(p_long_term)
    prob_original_list.append(p_original)    
    
    k = k + 1
#     if k % 1000 == 0:
#         print("elem = {}. k = {}.".format(elem, k))

In [36]:
len(f['h_' + str(elem_id)][()][0][0])

80

In [9]:
len(p5_satisfies)

580741

In [27]:
2232/580741*100

0.3843365631150547

In [10]:
df = pd.DataFrame({'p5_satisfies':p5_satisfies})
df.p5_satisfies.value_counts()

0    524370
1     56371
Name: p5_satisfies, dtype: int64

In [11]:
56371/df_tokens.shape[0]*100

9.706736738063956

# SMC

In [12]:
def get_nu_rho(n_satisfies, n_sampled_so_far, alpha, beta):
    ro = (n_satisfies + alpha) / (n_sampled_so_far + alpha + beta)
    nu = np.sqrt(((alpha + n_satisfies) * (n_sampled_so_far - n_satisfies + beta)) / (
            pow((alpha + n_sampled_so_far + beta), 2) * (alpha + n_sampled_so_far + beta + 1)))
    return ro, nu

In [14]:
# calculate time
elements_num = df_tokens.shape[0]
tmp = list(range(0, elements_num))
tokens_indx_rand = tmp.copy()
random.seed(42)
random.shuffle(tokens_indx_rand)
alpha = 1
beta = 1

n_satisfies_p5_list = []
n_sampled_so_far_list = []

smc_ro_estimates_p5 = []
smc_nu_estimates_p5 = []

n_satisfies_p5 = 0
n_sampled_so_far = 0

k = 0

for i in tokens_indx_rand:
    # TODO: vectorize this function using apply()
    # should be cumulative group by operation

    if p5_satisfies[i] == 1:
        n_satisfies_p5 = n_satisfies_p5 + 1

    n_sampled_so_far = n_sampled_so_far + 1
    rho_p5, nu_p5 = get_nu_rho(n_satisfies_p5, n_sampled_so_far, alpha, beta)

    n_satisfies_p5_list.append(n_satisfies_p5)
    n_sampled_so_far_list.append(n_sampled_so_far)

    smc_ro_estimates_p5.append(rho_p5)
    smc_nu_estimates_p5.append(nu_p5)
#     if k % 1000 == 0:
#         print(" k = {}.".format(k))
#     k = k + 1
    
_df_p5_smc = pd.DataFrame({'token_id': tokens_indx_rand,
                          'satisfies_p5': n_satisfies_p5_list,
                          'sampled': n_sampled_so_far_list,
                          'rho_p5': smc_ro_estimates_p5,
                          'nu_p5': smc_nu_estimates_p5
                          })

In [16]:
_df_p5_smc.shape

(580741, 5)

In [15]:
model_base_name = 'model_80_1_80_500'
verify_results_folder='E:/Dima/PhD/Papers/data/verify_results/'

In [17]:
_df_p5_smc.to_csv(verify_results_folder + "/" + model_base_name + "_p5_smc.csv", index=False)